In [2]:
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi

In [3]:
bd.projects.set_current(name='DOC_casestudies')

In [7]:
list(bd.databases)

['ecoinvent-3.8-biosphere',
 'ecoinvent-3.8-cutoff',
 'case_study_data',
 'test_db']

In [5]:
eidb = bd.Database("ecoinvent-3.8-cutoff")

In [61]:
for method in bd.methods:
    if 'IMPACT World+ World' in method[0]:
        #if 'Marine eutrophication' in method[2]:
            print(method)

('IMPACT World+ World', 'Midpoint', 'Freshwater ecotoxicity')
('IMPACT World+ World', 'Midpoint', 'Human toxicity cancer')
('IMPACT World+ World', 'Midpoint', 'Human toxicity non-cancer')
('IMPACT World+ World', 'Midpoint', 'Particulate matter formation')
('IMPACT World+ World', 'Midpoint', 'Marine eutrophication')
('IMPACT World+ World', 'Midpoint', 'Freshwater eutrophication')
('IMPACT World+ World', 'Midpoint', 'Terrestrial acidification')
('IMPACT World+ World', 'Midpoint', 'Freshwater acidification')
('IMPACT World+ World', 'Midpoint', 'Water scarcity')
('IMPACT World+ World', 'Midpoint', 'Land occupation, biodiversity')
('IMPACT World+ World', 'Midpoint', 'Land transformation, biodiversity')


In [53]:
# Getting an activity's name using its key
random_act = eidb.random()
#random_act.as_dict()

In [54]:
random_act

'market for wood preservation, oscillating pressure method, inorganic salt, containing Cr, outdoor use, ground contact' (kilogram, GLO, None)

In [47]:
for exc in random_act.exchanges():
    print(exc)

Exchange: 7100.0 cubic meter 'market for concrete, normal' (cubic meter, RoW, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 4000000.0 kilogram 'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 1.0 unit 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 100000000.0 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 700000.0 kilogram 'market for copper, cathode' (kilogram, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 100000000.0 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'mine infrastru

In [48]:
for exc in random_act.technosphere():
    print(exc)

Exchange: 7100.0 cubic meter 'market for concrete, normal' (cubic meter, RoW, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 4000000.0 kilogram 'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 100000000.0 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 700000.0 kilogram 'market for copper, cathode' (kilogram, GLO, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 100000000.0 megajoule 'market for heat, district or industrial, other than natural gas' (megajoule, Europe without Switzerland, None) to 'mine infrastructure construction, open cast, peat' (unit, NORDEL, None)>
Exchange: 5500.0 cubic meter 'market for plywood' (cubic meter, RER, None) to 'mine infrastructure constructio

In [55]:
for exc in random_act.biosphere():
    print(exc)

In [56]:
ef_gwp_key = [
    m for m in bd.methods if "climate change" == m[2] and "EF v3.0" == m[1]
].pop()
print(ef_gwp_key)

# We can look at the method details
ef_gwp_method = bd.Method(ef_gwp_key)
print(ef_gwp_method)

('ecoinvent-3.8', 'EF v3.0', 'climate change', 'global warming potential (GWP100)')
Brightway2 Method: ecoinvent-3.8: EF v3.0: climate change: global warming potential (GWP100)


In [57]:
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {random_act: 1},
    method=ef_gwp_key,
)
my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()
my_lca.score

3.9043795878924636

In [6]:
case_studies = bd.Database('test_db').register()

In [8]:
test_db = bd.Database("test_db")

In [22]:
for activity in test_db:
    print(activity['name'])

In [21]:
for activity in test_db:
    activity.delete()

In [23]:
new_activity = test_db.new_activity(
            code='test_activity_241205',  # Générer un code unique
            name='test_activity',
            unit="item",  # Unité en termes d'items
            type="process",  # Type d'activité
        )
        
# Sauvegarder l'activité
new_activity.save()


In [24]:
test_activity = [act for act in test_db if 'test_activity' in act['name']].pop()
test_activity

'test_activity' (item, GLO, None)

In [11]:
random_activity = eidb.random()
random_activity

'aircraft production, belly-freight aircraft, medium haul' (unit, GLO, None)

In [25]:
new_exchange = test_activity.new_exchange(
                    amount=15,
                    input=random_activity,
                    type='technosphere',#sub_exc['type'],
                    unit=random_activity['unit']
                )
new_exchange.save()

In [26]:
new_exchange = test_activity.new_exchange(
                    amount=1,
                    input=test_activity,
                    type='production',#sub_exc['type'],
                    unit=test_activity['unit']
                )
new_exchange.save()

In [27]:
for exc in test_activity.exchanges():
    print(exc.input['name'])

aircraft production, belly-freight aircraft, medium haul
test_activity


In [28]:
iww_methods = list(filter(lambda x: "IMPACT World+ World" in x[0], list(bd.methods)))
print("We have", len(iww_methods), "IMPACT World+ World methods we will evaluate.")

We have 11 IMPACT World+ World methods we will evaluate.


In [29]:
functional_units_1 = {
    "activity": {test_activity.id: 1},
}
config_1 = {
    "impact_categories": iww_methods
}
data_objs_1 = bd.get_multilca_data_objs(functional_units=functional_units_1, method_config=config_1)

In [30]:
mlca_1 = bc.MultiLCA(demands=functional_units_1, method_config=config_1, data_objs=data_objs_1)
mlca_1.lci()
mlca_1.lcia()
mlca_1.scores

{(('IMPACT World+ World', 'Midpoint', 'Freshwater ecotoxicity'),
  'activity'): 5338159917.427805,
 (('IMPACT World+ World', 'Midpoint', 'Human toxicity cancer'),
  'activity'): 1.3625324302835486,
 (('IMPACT World+ World', 'Midpoint', 'Human toxicity non-cancer'),
  'activity'): 4.440635502898678,
 (('IMPACT World+ World', 'Midpoint', 'Particulate matter formation'),
  'activity'): 32114.615982585907,
 (('IMPACT World+ World', 'Midpoint', 'Marine eutrophication'),
  'activity'): 5090.938644571369,
 (('IMPACT World+ World', 'Midpoint', 'Freshwater eutrophication'),
  'activity'): 14704.07213653994,
 (('IMPACT World+ World', 'Midpoint', 'Terrestrial acidification'),
  'activity'): 299534.18397185975,
 (('IMPACT World+ World', 'Midpoint', 'Freshwater acidification'),
  'activity'): 299534.18397185975,
 (('IMPACT World+ World', 'Midpoint', 'Water scarcity'),
  'activity'): 12928599.789348602,
 (('IMPACT World+ World', 'Midpoint', 'Land occupation, biodiversity'),
  'activity'): 450728.036

In [31]:
new_activity = test_db.new_activity(
            code='test_subactivity_241205',  # Générer un code unique
            name='test_subactivity',
            unit="item",  # Unité en termes d'items
            type="process",  # Type d'activité
        )
        
# Sauvegarder l'activité
new_activity.save()


In [32]:
test_subactivity = [act for act in test_db if 'test_subactivity' in act['name']].pop()
test_subactivity

'test_subactivity' (item, GLO, None)

In [33]:
random_activity_2 = eidb.random()
random_activity_2

'photovoltaic mounting system production, for flat-roof installation' (square meter, RoW, None)

In [34]:
new_exchange = test_subactivity.new_exchange(
                    amount=15,
                    input=random_activity_2,
                    type='technosphere',#sub_exc['type'],
                    unit=random_activity['unit']
                )
new_exchange.save()

In [35]:
new_exchange = test_subactivity.new_exchange(
                    amount=1,
                    input=test_subactivity,
                    type='production',#sub_exc['type'],
                    unit=test_subactivity['unit']
                )
new_exchange.save()

In [37]:
for exc in test_subactivity.exchanges():
    print(exc.input['name'])

photovoltaic mounting system production, for flat-roof installation
test_subactivity


In [38]:
new_exchange = test_activity.new_exchange(
                    amount=3,
                    input=test_subactivity,
                    type='technosphere',#sub_exc['type'],
                    unit=test_subactivity['unit']
                )
new_exchange.save()

In [39]:
for exc in test_activity.exchanges():
    print(exc.input['name'])

aircraft production, belly-freight aircraft, medium haul
test_activity
test_subactivity


In [41]:
functional_units_1 = {
    "activity": {test_activity.id: 1},
}
config_1 = {
    "impact_categories": iww_methods
}
data_objs_1 = bd.get_multilca_data_objs(functional_units=functional_units_1, method_config=config_1)

In [42]:
mlca_1 = bc.MultiLCA(demands=functional_units_1, method_config=config_1, data_objs=data_objs_1)
mlca_1.lci()
mlca_1.lcia()
mlca_1.scores

{(('IMPACT World+ World', 'Midpoint', 'Freshwater ecotoxicity'),
  'activity'): 5338343831.987313,
 (('IMPACT World+ World', 'Midpoint', 'Human toxicity cancer'),
  'activity'): 1.3626457554321096,
 (('IMPACT World+ World', 'Midpoint', 'Human toxicity non-cancer'),
  'activity'): 4.440969577394308,
 (('IMPACT World+ World', 'Midpoint', 'Particulate matter formation'),
  'activity'): 32115.831121607524,
 (('IMPACT World+ World', 'Midpoint', 'Marine eutrophication'),
  'activity'): 5091.108456793461,
 (('IMPACT World+ World', 'Midpoint', 'Freshwater eutrophication'),
  'activity'): 14704.099051467287,
 (('IMPACT World+ World', 'Midpoint', 'Terrestrial acidification'),
  'activity'): 299542.6123288315,
 (('IMPACT World+ World', 'Midpoint', 'Freshwater acidification'),
  'activity'): 299542.6123288315,
 (('IMPACT World+ World', 'Midpoint', 'Water scarcity'),
  'activity'): 12929324.736814499,
 (('IMPACT World+ World', 'Midpoint', 'Land occupation, biodiversity'),
  'activity'): 450749.4638